# Advanced Tutorial: Named Entity Recognition and the Bi-LSTM Conditional Random Field Algorithm

Tutorial Link:  https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

A bit more basic is the Bi-Directional Recurrent Neural Network ([Ref]()):

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/bilstm_flow.png" width="50%">

It does however show sequences that are embedded used to make predictions.  They are trained on both past as well as future information from the given data.  This will make more sense shortly.

In [ ]:
import sys
{sys.executable}

In [ ]:
! {sys.executable} -m pip install jdc bs4 requests

In [ ]:
import torch
torch.__version__

In [ ]:
from bs4 import BeautifulSoup
import requests
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import jdc

torch.manual_seed(1)

## Outline

* Definitions

    * BLSTM
    * CRF and potentials
    * Viterbi

* Helper Functions

* Create the Network

* Train

* Evaluate

* Exercise

## Definitions


### BLSTM (Bi-Long Short-Term Memory)

As we saw, an LSTM addresses the vanishing gradient problem of the generic RNN by adding cell state and more non-linear activation function layers to pass on or attenuate signals to varying degrees.  However, the main limitation of an LSTM is that it can **only account for context from the past**, that is, the hidden state, h_t, takes only past information as input.

For the task of **Named Entity Recognition (NER)** is helpful to have context from past as well as the future, or left and right contexts.  This can be addressed with a BLSTM which is two LSTMs, one processing information in a forward fashion as we have already seen and another LSTM that processes the sequences in a reverse fashion giving the future context.  _That second LSTM is just reading the sentence in reverse._  

The hidden states from both LSTMs are then concatenated into a final output layer or vector.

### Conditional Random Field

We don't have to stop at the output vector from the BLSTM!  We're not to our tag for the entity, yet.  We need to understand costs of moving from one tag to the next (or staying put on a tag, even).

In a CRF, the parameters are simply the costs associated with transitioning from one tag to another in a sequence.  They are stored as a _transition matrix_ as is shown below.

Say **B** is the tag for the beginning of a multi-word entity, **I** signifies that we are inside an entity (which could be one word) and **O** means we are outside an entity. 

Here's an example of B-I-O schema labeling (by the way, there are a myriad of other schemas out there - see [Referenes](#references) for some more).

| Word | Schema Tag |
| --- | --- |
| Micheleen | I |
| was | O |
| born | O |
| in | O |
| North | B |
| Carolina | I |
| but | O |
| grew | O |
| up | O |
| in | O |
| Texas | I |


Let's look at an example transition matrix for the costs of moving from one tag to the next (remember our BLSTM is understanding both the forward and reverse ordering to get more accurate boundaries for the named entities).

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/fbc7992e8c9399bb713bbf63664bf34b13f091c5/images/crf_transition_matrix.png" width="60%">

### Viterbi Algorithm

The Viterbi algorithm, by finding the optimal path, can figure out for us what the final tag should be for the input.  We decode the results from the CRF and, thus, it is often seen as the final step of a CRF.

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/viterbi.png" width="70%">


### Putting it All Together

Here is an example from biomedical research text.

Here we have word embeddings as the data for the forward and reverse LSTMs.  The resulting forward vector (V_f) and backwards vector (V_b) are concatenated into a final vector (V_o) that feeds into the CRF layer and is decoded via the Viterbi algorithm into the optimal tag for that input.

<br><br>

![Bi LSTM detail](https://ars.els-cdn.com/content/image/1-s2.0-S1532046417300977-gr4.jpg)

<div  align="right"><a href="https://www.sciencedirect.com/science/article/pii/S1532046417300977">Reference</a></div>

### Label small sample of data

Label Meanings

* B stands for 'beginning' (signifies beginning of an NE)
* I stands for 'inside' (signifies that the word is inside an NE)
* O stands for 'outside' (signifies that the word is just a regular word outside of an NE)

More information on label schemas [here](http://natural-language-understanding.wikia.com/wiki/Named_entity_recognition)

## Helper Functions

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
# https://pytorch.org/docs/master/tensors.html#torch.Tensor.expand
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

## Create the Network

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim # represents both directions
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.  This is how the CRF stores
        # costs.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag in the
        # transition matrix of the CRF
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

In [ ]:
%%add_to BiLSTM_CRF

def init_hidden(self):
    """Two tensors to hold hidden states, one for each
    LSTM direction with dimensions of (num_layers, 
    minibatch, hidden_dim)"""
    # Minibatch small bc small dataset below
    return (torch.randn(2, MINIBATCH_SIZE, self.hidden_dim // 2),
            torch.randn(2, MINIBATCH_SIZE, self.hidden_dim // 2))


In [ ]:
%%add_to BiLSTM_CRF


def _forward_backwards_trick(self, feats):
    """Main part of the Conditional Random Field.  
    
    
    Input:
        The word embeddeding vectors for a sentence
    
    Since we’re using PyTorch to compute gradients for us, 
    we technically only need the forward part of the forward-backward 
    algorithm """
    # Do the forward algorithm to compute the partition function
    # The "full" function fills in with a default value
    # Here we create a 1x3 matrix
    init_alphas = torch.full((1, self.tagset_size), -10000.)
    
    # init_alphas holds costs of moving from the START_TAG
    # to B, I or O
    # Let's set the cost of staying at the start tag
    # to a low cost.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(self.tagset_size):
            # broadcast (copy into new dim) the emission score: 
            # it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, self.tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = self.transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1) # torch.cat TODO
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

In [ ]:
%%add_to BiLSTM_CRF

def _get_lstm_features(self, sentence):
    """Compute output vector of BLSTM"""
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

In [ ]:
%%add_to BiLSTM_CRF

def _score_sentence(self, feats, tags):
    """"""
    # Gives the score of a provided tag sequence
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
        score = score + \
            self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

In [ ]:
%%add_to BiLSTM_CRF

def _viterbi_decode(self, feats):
    """Implements Viterbi algorithm for finding most likely sequence of labels.

    We take the maximum over the previous states as opposed to the sum. 
    Input:
        loglikelihoods: torch tensor.
    Output:
        tuple. The first entry is the loglikelihood of this sequence. The second is 
        the most likely sequence of labels. 
    """
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(self.tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

In [ ]:
%%add_to BiLSTM_CRF

def neg_log_likelihood(self, sentence, tags):
    """"""
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_backwards_trick(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

In [ ]:
%%add_to BiLSTM_CRF

def forward(self, sentence):  # dont confuse this with _forward_backwards_trick above.
    """"""
    # Get the emission scores (hidden layer vector outputs) from the 
    # BiLSTM 
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

## Train

SGD weight_decay

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
MINIBATCH_SIZE = 1

In [ ]:
# Make up some training data - this is from a NYTimes recipe
# The ingrediants and equipment are labeled as entities here.
training_data = [(
    "strain broth through a fine sieve (or a colander lined \
         with cheesecloth) into a separate container.".split(),
    "O I O O O I O O I O O I O O B I".split()
), (
    "pour broth into pot with vegetables and heat to a simmer.".split(),
    "O I O I O I O O O O O".split()
)]

In [ ]:
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [ ]:
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

In [ ]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
# again, normally you would do more than 300 epochs, but we have
# toy data
for epoch in range(300):  
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. A lot happens.  Run our forward pass to get features from BLSTM,
        # run the CRF and get the negative log likelihoods and find the best 
        # "path" through sentence with the tags using the viterbi algorithm 
        # (also part of forward pass).
        # BTW our dynamic computational graph is created with the forward pass
        # Returns the forward score - ground truth score (our loss measure)
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients (backprop), and update the 
        # parameters by calling optimizer.step() - optimizer here is 
        # SGD for our CRF
        loss.backward()
        optimizer.step()

In [ ]:
# Check predictions after training
# No need to accumulate gradients because this is a validation
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
    print(training_data[0][0])
# We got it!

## Evaluate

Let's test our model on an unseen sentence.



In [ ]:
test_data = [(
    "serve immediately, in a tureen or from the pot, sprinkling \
        each serving with herbs".split(),
    "O O O O I O O O I O O I O I".split()
)]

word_to_ix = {}
for sentence, tags in test_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

In [ ]:
with torch.no_grad():
    precheck_sent = prepare_sequence(test_data[0][0], word_to_ix)
    print(model(precheck_sent))
    print(test_data[0][1])
    print(test_data[0][0])

## Exercise

Use BeautifulSoup Python library and the Requests library to download a recipe of your choosing from https://cooking.nytimes.com/ and find it's steps section.


To get started, you can:

```python
page = requests.get('')
soup = BeautifulSoup(page.content, 'html.parser')
steps = soup.findAll("ol", {"class": "recipe-steps"})
```

Instructions:
1. Clean up the html tags and any strange characters.
2. Label entities with YEDDA:  https://github.com/michhar/YEDDA.
3. Save the labels as text and write code to identify entities as labels for the BiLSTM above.
4. Run a BiLSTM on this data and validate the predictions against the original labels.
5. Run some test data through your model.

## References

1. [Understanding Bidirectional RNN in PyTorch](https://towardsdatascience.com/understanding-bidirectional-rnn-in-pytorch-5bd25a5dd66)
2. [Conditional Random Field Tutorial in PyTorch](https://towardsdatascience.com/conditional-random-field-tutorial-in-pytorch-ca0d04499463)
3. [Character-level neural network for biomedical named entity recognition](https://www.sciencedirect.com/science/article/pii/S1532046417300977)
4.  [Other named entity tag schemas](https://lingpipe-blog.com/2009/10/14/coding-chunkers-as-taggers-io-bio-bmewo-and-bmewo/)

In [2]:
from IPython.display import HTML

HTML('''
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the hint."></form>''')

In [ ]:
# HINT!  (cleaning text data)
import requests
from bs4 import BeautifulSoup

page = requests.get('https://cooking.nytimes.com/recipes/1018442-chicken-soup-from-scratch')
soup = BeautifulSoup(page.content, 'html.parser')
steps = soup.findAll("ol", {"class": "recipe-steps"})

import re

def cleanhtml(raw_html):
    """Function to clean up the html tags in data."""
    cleanr = re.compile('<.*?>')
    # Remove html tags
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', ' ').rstrip().strip()
    # Remove special quotes
    cleantext = cleantext.replace('“', '').replace('”', '')
    cleantext = cleantext.lower()
    return cleantext

cleansteps = cleanhtml(str(steps[0]))
cleansteps

with open('sample_data.txt', 'w') as f:
    f.write(cleansteps)

In [ ]:
# This data is for updating training data or to use in the exercise
with open('sample_data.txt', 'r') as f:
    data = f.read()
print(data)